# Q-Learning Mouse
Mouse tries to find the cheese located in the bottom-right cell of the puzzle. While trying to find the cheese, the mouse has to avoid being eaten by cats whose locations you can choose (instructions given bellow). Cats are _stationary_, as in _located in the cells which the mouse should not enter_. Mouse learns how to guide his way through the puzzle and get the beloved cheese by applying the Q-Learning algorithm. Q-Learning algorithm is a Reinforcement Learning algortihm which is model-free - meaning that it doesn't need to know (the model of) the environment.

## The Q-Learning algorithm applied by the mouse
Mouse learns by trial and error. At first, he wanders arround; He enters a __cell__ without any __knowledge__ of the locations of the cats or the cheese (These entities are bolded for a reason; They're vital in the Q-Learning algorithm). However, as he stumbles upon the first cat in his wanderings, he gets his first __reward__! He gets eaten by the cat! 

However, this __reward__ is a negative one - it serves the mouse as an indicator that his __action of translating from the previous cell to the one where the cat is located is BAD__! Mouse learns from this and _remembers_ (in __the knowledge__ of the locations of the cats and the cheese) not to take that action ever again. So far we have these entities:
* _A cell_: an entity that mouse can enter or leave.
* _The knowledge_: mouse's view of the puzzle - his assumptions of which cell to choose as his next one so he does not get eaten by the cats and to be one step closer to the cheese. For now we know that one action is a very negative one!
* _A reward_: something he gets for transitioning from a cell to another cell; For now, all we know is that it is negative when he gets eaten! 

After getting resurrected (yes, pls don't lose me) and starting his voyage from the starting position again, he is now a bit smarter! However, while wandering around he stumbles upon another cat and gets eaten again! Nevermind, the process continues. After a few iterations, __mouse takes an action which leads him to the cheese__. He gets a huge positive reward!
So now, we have these entities:
* _A cell_:
* _The knowledge_: For now, we know that there are a few actions which are very negative and one action that is positive!
* _A reward_: Now we know that it is negative when he gets eaten and positive when he gets the cheese.

At this stage it is important to expose a subtlety in the algorithm which makes a huge difference! Namely, picture this: The mouse is residing at a cell $C$. He wants to know which action $A_i$ is the best to take, as in: which cell $C_i'$ should be the best to reside at, next. He is in the process of picking the cell $C'_i$, of course, from the neighbouring cells (denoted as $C'_i \in \text{neighbouring cells}(C)$) of the current cell $C$. However, if you remember, __even at the cell $C$ he knows how sound are the actions to take at any of the neighbouring cells $C'_i$ or at any cell for that matter__. And, he takes that into account! He takes the action $A_i$ and gets to the cell $C'_i$ from which, afterwards, he can take the best valued actions.

So now, his choice of an action $A$ also depends on how sound will the actions in the future be! That is the essence of the Q-Learning algorithm. In that way, his choice of even the first action to take, or any for that matter, is influenced by the future expectations of whether he'll get the cheese or get eaten by choosing that action.

After a finite number of iterations, the process terminates. __Mouse has found the best path to take to get to the cheese. We can construct that path just by choosing the best valued action $A_i$ to transition from a given state $C$ to a neighbouring state $C'_i$__. The path which won't expose him to cats. 

| Mentioned entities 	|    Reinforcement Learning terminology   	|
|:------------------:	|:---------------------------------------:	|
|       A cell       	|                 A state                 	|
|    The knowledge   	| Q-function with the state-action values 	|
|      A reward      	|                 A reward                	|

---

### Setting up the Environment
Environment contains/represents:
1. The size of the puzzle which the mouse will be exploring. 
2. The starting positon of the mouse in the puzzle.
3. The locations of the cats in the puzzle. <p></p>
__You can change these attributes at the indicated spots bellow__.

In [1]:
from td_exercise_environment import Environment
from bokeh.io import output_notebook, show
from bokeh.plotting import figure
output_notebook()


# 1. The size of the puzzle 
puzzle_width = 12
puzzle_height = 12

# 2. The starting position of the mouse
start_x = 11
start_y = 0


env = Environment(puzzle_height, puzzle_width)
env.set_start_state(start_x, start_y)

Loading BokehJS ...

In [2]:
import numpy as np
cats = []

'''
# Relevant only if the positions of the cats are chosen at random 
# (comment out the cat appending block bellow and uncomment this if you wish)
    
no_cats = 18    
for i in range(no_cats):
    cats.append((np.random.randint(0, puzzle_width), np.random.randint(0, puzzle_height)))
'''

# 3. The locations of the cats in the puzzle.
cats.append((0, 2))
cats.append((1, 1))
cats.append((1, 3))
cats.append((2, 1))
cats.append((3, 1))
cats.append((3, 3))
cats.append((4, 3))
cats.append((4, 4))
cats.append((4, 5))
cats.append((4, 6))
cats.append((5, 2))
cats.append((5, 3))
for i in range(5, 12): cats.append((i, 1))
for i in range(6, 12): cats.append((6, i))
for i in range(3, 12): cats.append((10, i))

env.set_cats(cats)

### Drawing the Environment
It's time to draw the environment you specified! After you run the cell, check bellow whether the drawn environment matches the one you specified.

In [3]:
from bokeh.models.glyphs import Rect

rect_lst = []
WIDTH=1
HEIGHT=1

def draw_rectangles(p, env):
    '''
    Meaning of the colors:
        Red:    Cat is located in this cell. From the mouse's perspective,
                he does not know that.
        Green:  A cell which the mouse has taken to be on his path to the
                cheese. This will be known only after the Q-Learning
                algorithm has terminated.
        Blue:   Starting state and the state in which the cheese is located.
                Mouse is also not aware of them.
    '''
    for i in range(env._size_y):
        row = []
        for j in range(env._size_x):
            rect = Rect(x=j, y=-i, width=WIDTH, height=HEIGHT, 
                        fill_color="#CAB2D6", line_width=1.0, line_color = "#000000")
            p.add_glyph(rect)
            row.append(rect)
        rect_lst.append(row)
    for cat in env._cats:
        rect_lst[cat[1]][cat[0]].fill_color="#ff0000"
    rect_lst[env._start_state._y][env._start_state._x].fill_color="#0015ff"
    rect_lst[env._size_y-1][env._size_x-1].fill_color="#0015ff"
                        
def update_rects(Q, env):
    '''
    Updates the rectangles - colors them according to the context.
    '''
    previous_states = []
    state = env.reset()
    done = False
    while not done:
        if state in previous_states:
            print("Mouse is going in circles.")
            print("Mouse cannot get to the cheese.")
            break
        previous_states.append(state)
        rect_lst[state._y][state._x].fill_color="#69ea69"
        best_next_action = max(a.Q[state].items(), key=operator.itemgetter(1))[0]
        next_state, _, done = env.step(best_next_action)
        state = next_state
    
p = figure()
draw_rectangles(p, env)
p.axis.visible = False

show(p)

### Let the mouse find the cheese
The main Q-Learning loop. After you run this cell, check whether the mouse has gotten the cheese!

In [4]:
from td_exercise_agent import get_random_action, make_epsilon_greedy_policy, Agent
from random import random
import operator

# Settings for the Q-Learning algorithm
num_episodes=500
epsilon=0.2          # Probability of choosing a random action
discount_factor=0.95  # Mouse's willingness to wait for a distant reward
alpha=0.5            # Learning rate

# Agent represents a mouse
a = Agent(env)

# Set the beginning value of every action to be 0 - 
# Mouse doesn't know anything about the environment
a.init_q_values()

# This will be explained later
policy = make_epsilon_greedy_policy(a.Q, epsilon)

# Run a finite number of iterations - episodes
for i in range(num_episodes):

    # Initialize state - put the mouse in his starting position
    state = env.reset()

    # Let the mouse explore the environment in this iteration
    while True:
        '''
        Choose an action A to take from a state S by following a policy.
        In the explanation above, the policy meant to take the best-valued
        action at every state. However, in reality, mouse needs to take a 
        random action at some probability. Mouse does this in order to discover 
        a potentially better path. He is greedy to an extent!
        '''
        action_probs = policy(state)
        action = get_random_action(action_probs)

        # Take an action in the environment - mouse goes to the next cell.
        next_state, reward, done = env.step(action)
            
        '''
        Evaluate the taken action by taking various things into account:
        1) If the action has led to a next_state in which the best_next_action
        is e.g. really good -> then the taken action is also really good!
        2) If the reward for getting into this cell is e.g very negative
        (as in the mouse got eaten) -> then the taken action is very bad!
        +) the defined constants
        '''
        best_next_action = max(a.Q[next_state].items(), key=operator.itemgetter(1))[0]
        td_target = reward + discount_factor * a.Q[next_state][best_next_action]
        td_delta = td_target - a.Q[state][action]
        a.Q[state][action] += alpha * td_delta

        # Check if the current state is terminal.
        if done:
            break

        state = next_state
        
# Color the according cells green.
update_rects(a.Q, env)

show(p)

#### Author: Bojan Poprzen, 2019.